In [1]:
import requests

import time
from requests.exceptions import RequestException

def fetch_data_notebook(url, retries=3, delay=2):
    """
    Fetches data from the given URL with retry and error handling for Jupyter Notebook.

    :param url: The URL to fetch data from.
    :param retries: Number of retries in case of failure.
    :param delay: Delay (in seconds) between retries.
    :return: Response object if successful, or None if all retries fail.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://google.com'
    }

    for attempt in range(retries):
        try:
            print(f"Fetching data from: {url} (Attempt {attempt + 1}/{retries})")
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                print("Data fetched successfully!")
                return response
            else:
                print(f"Unexpected status code {response.status_code}. Retrying...")
        except RequestException as e:
            print(f"Error: {e}. Retrying...")

        time.sleep(delay)
    
    print("Failed to fetch data after multiple retries.")
    return None

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = fetch_data_notebook(standings_url)#requests.get(standings_url)

Fetching data from: https://fbref.com/en/comps/9/Premier-League-Stats (Attempt 1/3)
Data fetched successfully!


In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


In [5]:
years = list(range(2024, 2022, -1))
print(years)
all_matches = []

[2024, 2023]


In [6]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
counter = 1

In [7]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    print(previous_season)
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        print(f'now is {counter}')
        counter+=1
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        except Exception:
            continue
            
        print(shooting)
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

/en/comps/9/2023-2024/2023-2024-Premier-League-Stats
now is 1


   For Liverpool                                                           \
            Date   Time            Comp           Round  Day Venue Result   
0     2024-08-17  12:30  Premier League     Matchweek 1  Sat  Away      W   
1     2024-08-25  16:30  Premier League     Matchweek 2  Sun  Home      W   
2     2024-09-01  16:00  Premier League     Matchweek 3  Sun  Away      W   
3     2024-09-14  15:00  Premier League     Matchweek 4  Sat  Home      L   
4     2024-09-17  21:00    Champions Lg    League phase  Tue  Away      W   
5     2024-09-21  15:00  Premier League     Matchweek 5  Sat  Home      W   
6     2024-09-25  20:00         EFL Cup     Third round  Wed  Home      W   
7     2024-09-28  17:30  Premier League     Matchweek 6  Sat  Away      W   
8     2024-10-02  20:00    Champions Lg    League phase  Wed  Home      W   
9     2024-10-05  12:30  Premier League     Matchweek 7  Sat  Away      W   
10    2024-10-20  16:30  Premier League     Matchweek 8  Sun  Home      W   

now is 2


   For Arsenal                                                                \
          Date   Time            Comp           Round  Day Venue Result   GF   
0   2024-08-17  15:00  Premier League     Matchweek 1  Sat  Home      W  2.0   
1   2024-08-24  17:30  Premier League     Matchweek 2  Sat  Away      W  2.0   
2   2024-08-31  12:30  Premier League     Matchweek 3  Sat  Home      D  1.0   
3   2024-09-15  14:00  Premier League     Matchweek 4  Sun  Away      W  1.0   
4   2024-09-19  21:00    Champions Lg    League phase  Thu  Away      D  0.0   
5   2024-09-22  16:30  Premier League     Matchweek 5  Sun  Away      D  2.0   
6   2024-09-25  19:45         EFL Cup     Third round  Wed  Home      W  5.0   
7   2024-09-28  15:00  Premier League     Matchweek 6  Sat  Home      W  4.0   
8   2024-10-01  20:00    Champions Lg    League phase  Tue  Home      W  2.0   
9   2024-10-05  15:00  Premier League     Matchweek 7  Sat  Home      W  3.0   
10  2024-10-19  17:30  Premier League   

now is 3


...

   For Luton Town                                                         \
             Date   Time            Comp                Round  Day Venue   
0      2023-08-12  15:00  Premier League          Matchweek 1  Sat  Away   
1      2023-08-25  20:00  Premier League          Matchweek 3  Fri  Away   
2      2023-08-29  19:45         EFL Cup         Second round  Tue  Home   
3      2023-09-01  20:00  Premier League          Matchweek 4  Fri  Home   
4      2023-09-16  15:00  Premier League          Matchweek 5  Sat  Away   
5      2023-09-23  15:00  Premier League          Matchweek 6  Sat  Home   
6      2023-09-26  19:45         EFL Cup          Third round  Tue  Away   
7      2023-09-30  15:00  Premier League          Matchweek 7  Sat  Away   
8      2023-10-03  19:30  Premier League          Matchweek 2  Tue  Home   
9      2023-10-07  12:30  Premier League          Matchweek 8  Sat  Home   
10     2023-10-21  15:00  Premier League          Matchweek 9  Sat  Away   
11     2023-

now is 39


   For Burnley                                                               \
          Date   Time            Comp               Round  Day Venue Result   
0   2023-08-11  20:00  Premier League         Matchweek 1  Fri  Home      L   
1   2023-08-27  14:00  Premier League         Matchweek 3  Sun  Home      L   
2   2023-08-30  19:45         EFL Cup        Second round  Wed  Away      W   
3   2023-09-02  15:00  Premier League         Matchweek 4  Sat  Home      L   
4   2023-09-18  19:45  Premier League         Matchweek 5  Mon  Away      D   
5   2023-09-23  20:00  Premier League         Matchweek 6  Sat  Home      L   
6   2023-09-26  19:45         EFL Cup         Third round  Tue  Away      W   
7   2023-09-30  15:00  Premier League         Matchweek 7  Sat  Away      L   
8   2023-10-03  19:30  Premier League         Matchweek 2  Tue  Away      W   
9   2023-10-07  15:00  Premier League         Matchweek 8  Sat  Home      L   
10  2023-10-21  15:00  Premier League         Matchw

now is 40


   For Sheffield United                                                   \
                   Date   Time            Comp                Round  Day   
0            2023-08-12  15:00  Premier League          Matchweek 1  Sat   
1            2023-08-18  19:45  Premier League          Matchweek 2  Fri   
2            2023-08-27  14:00  Premier League          Matchweek 3  Sun   
3            2023-08-30  19:45         EFL Cup         Second round  Wed   
4            2023-09-02  12:30  Premier League          Matchweek 4  Sat   
5            2023-09-16  15:00  Premier League          Matchweek 5  Sat   
6            2023-09-24  16:30  Premier League          Matchweek 6  Sun   
7            2023-09-30  15:00  Premier League          Matchweek 7  Sat   
8            2023-10-07  15:00  Premier League          Matchweek 8  Sat   
9            2023-10-21  20:00  Premier League          Matchweek 9  Sat   
10           2023-10-28  15:00  Premier League         Matchweek 10  Sat   
11          

In [8]:
len(all_matches)

40

In [9]:
match_df = pd.concat(all_matches)

In [10]:
match_df.columns = [c.lower() for c in match_df.columns]

In [11]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Match Report,NaN,18.0,5.0,14.8,0.0,0,0,2024,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,Match Report,NaN,19.0,8.0,13.6,1.0,0,0,2024,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,Match Report,NaN,11.0,3.0,13.4,0.0,0,0,2024,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,14.0,5.0,14.9,0.0,0,0,2024,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,Match Report,NaN,19.0,12.0,16.6,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2023,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2023,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2023,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2023,Sheffield United


In [12]:
match_df.to_csv("matches2.csv")